# 🐍 파이썬 텍스트 분석: Chapter 6. Kiwipiepy와 최신 AI를 활용한 감성 분석

이 노트북에서는 미리 정답(레이블)이 정해진 데이터를 사용하여 모델을 학습시키는 **지도 학습(Supervised Learning)**을 텍스트 분석에 적용하는 심화 방법을 배웁니다.

특히, 기존의 형태소 분석기에서 더 빠르고 발전된 **Kiwipiepy**를 사용하여 텍스트를 전처리하고, 두 가지 다른 접근 방식으로 감성 분석 모델을 구축합니다.

1.  **전통적인 머신러닝 방식**: `Kiwipiepy`로 전처리한 텍스트를 `TF-IDF`라는 기법으로 벡터화하여, 로지스틱 회귀 모델을 학습시킵니다.
2.  **최신 AI 임베딩 방식**: `OpenAI`의 강력한 언어 모델을 사용하여 텍스트를 의미가 풍부한 벡터(임베딩)로 변환하고, 이를 기반으로 모델을 학습시켜 성능을 비교합니다.

이 과정을 통해 수만 개의 영화 리뷰 텍스트를 이용해 해당 리뷰가 '긍정'적인지 '부정'적인지를 예측하는 감성 분석 모델을 직접 만들어보겠습니다.

---

### 💡 시작 전 준비: 라이브러리 설치 및 데이터 로드

이번 실습에서는 먼저 `kiwipiepy`와 `openai` 라이브러리를 설치해야 합니다. `kiwipiepy`는 한국어 텍스트를 효과적으로 처리하기 위한 형태소 분석기이며, `openai`는 텍스트 임베딩을 위해 사용됩니다.

데이터셋은 이전과 동일하게 대표적인 한국어 감성 분석 데이터셋인 **NSMC(Naver Sentiment Movie Corpus)** 를 사용합니다. 이 데이터셋은 네이버 영화의 리뷰 20만 개와 각 리뷰에 대한 긍정(레이블 1) 또는 부정(레이블 0) 평가를 포함하고 있습니다.

In [ ]:
# 필요 라이브러리 설치
!pip install kiwipiepy pandas scikit-learn plotly openai tqdm

In [34]:
import pandas as pd
import numpy as np
import re
import os
from kiwipiepy import Kiwi
import openai
from tqdm.auto import tqdm

# tqdm을 pandas에 적용하기 위한 설정
tqdm.pandas()

In [35]:
# 1. 데이터 다운로드 및 로드
# 도커/로컬 환경에 따라 경로는 수정하셔야 합니다.
train_df = pd.read_csv("../datasets/text/nsmc/ratings_train.txt", sep='\t')
test_df = pd.read_csv("../datasets/text/nsmc/ratings_test.txt", sep='\t')

In [36]:
# 2. 데이터 정제 (결측치 및 중복 제거)
train_df = train_df.dropna().drop_duplicates(subset=['document'])
test_df = test_df.dropna().drop_duplicates(subset=['document'])

In [37]:
# 3. 실습을 위해 데이터 샘플링
train_df = train_df.sample(n=1000, random_state=42)
test_df = test_df.sample(n=250, random_state=42)

In [ ]:
train_df.head()

In [39]:
from tqdm import tqdm

In [ ]:
# 4. Kiwipiepy를 사용한 텍스트 전처리 함수
kiwi = Kiwi()
# 파생 명사, 파생 부사 등을 더 잘게 분리하는 옵션 추가
# 예: '고마움' -> '고맙/VA-I', '음/ETN'
def kiwi_preprocess(text):
    text = re.sub(r'[^가-힣\s]', '', str(text)) # 한글과 공백만 남기기
    tokens = kiwi.tokenize(text, split_complex=True)
    # CountVectorizer/TfidfVectorizer 입력을 위해 공백으로 구분된 문자열 반환
    return ' '.join([token.form for token in tokens])

# 전처리 적용 (Kiwipiepy는 C++로 구현되어 있어 속도가 빠릅니다)
train_df['document_processed'] = train_df['document'].progress_apply(kiwi_preprocess)
test_df['document_processed'] = test_df['document'].progress_apply(kiwi_preprocess)

print("데이터 준비 완료!")
train_df.head()

---

## Part 1. TF-IDF를 이용한 전통적 감성 분석

### 1. 특징 벡터화와 데이터 분리 (Feature Vectorization & Data Splitting)

#### 💡 개념 (Concept)

모델 학습을 위해서는 텍스트 데이터를 숫자 벡터로 변환해야 합니다. 첫 번째 접근법으로, 단어의 빈도와 문서에서의 중요도를 함께 고려하는 **TF-IDF 방식**을 사용하겠습니다.

또한, 모델의 성능을 객관적으로 평가하기 위해 전체 데이터를 **학습 데이터(Training Data)**와 **테스트 데이터(Test Data)**로 분리해야 합니다. 모델은 학습 데이터를 보고 패턴을 익히며, 이전에 한 번도 본 적 없는 테스트 데이터를 얼마나 잘 맞추는지를 통해 성능을 평가받게 됩니다. 이는 모델이 단순히 데이터를 암기하는 것(과적합, Overfitting)이 아니라, 일반화된 예측 능력을 갖췄는지 확인하기 위함입니다.

#### 💻 예시 코드 (Example Code)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# 데이터셋을 특징(X)과 레이블(y)로 분리
X_train = train_df['document_processed']
y_train = train_df['label']
X_test = test_df['document_processed']
y_test = test_df['label']

# TF-IDF 벡터화
# 학습 데이터에 대해서는 fit과 transform을 모두 수행 (fit_transform)
# 테스트 데이터에 대해서는 학습된 어휘사전을 기준으로 transform만 수행
tfidf_vectorizer = TfidfVectorizer(min_df=3, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("TF-IDF 벡터화 완료.")
print(f"학습 데이터 (TF-IDF) 형태: {X_train_tfidf.shape}")
print(f"테스트 데이터 (TF-IDF) 형태: {X_test_tfidf.shape}")

#### ✏️ 연습 문제 (Practice Problems)

1.  `TfidfVectorizer`를 생성할 때, `max_features=2000` 파라미터를 추가하여 가장 빈도가 높은 2000개의 단어(n-gram 포함)만 사용하도록 설정해 보세요. TF-IDF 행렬의 형태(`shape`)가 어떻게 변하는지 확인해 보세요.

In [42]:
# 코드 작성

2.  `kiwipiepy`의 `tokenize` 메소드에는 `split_complex` 외에도 `normalize_coda=True`와 같은 유용한 옵션이 있습니다. 이 옵션은 '먹었엌ㅋㅋ'와 같이 어미 뒤에 자음이 붙는 신조어를 '먹었어'와 'ㅋㅋㅋ'로 분리해줍니다. `kiwi_preprocess` 함수에 이 옵션을 추가하고 전처리를 다시 실행한 뒤, TF-IDF 행렬의 크기가 어떻게 변하는지 관찰해보세요.

In [ ]:
# 코드 작성


---

### 2. 머신러닝 모델 학습 및 예측 (Model Training & Prediction)

#### 💡 개념 (Concept)

이제 준비된 숫자 데이터(TF-IDF 벡터)를 머신러닝 모델에 입력하여 학습시킬 차례입니다. 텍스트 분류 문제에서 준수한 성능을 보이는 **로지스틱 회귀(Logistic Regression)** 모델을 사용하겠습니다. `fit` 메소드를 사용하여 모델은 TF-IDF 벡터(특징)와 감성 레이블(정답) 사이의 관계를 학습합니다. 학습이 완료된 후에는 `predict` 메소드를 사용하여 새로운 데이터의 정답을 예측할 수 있습니다.

#### 💻 예시 코드 (Example Code)

In [ ]:
from sklearn.linear_model import LogisticRegression

# 1. 로지스틱 회귀 모델 생성 및 학습
lr_model = LogisticRegression(random_state=42, C=5, max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)
print("로지스틱 회귀 모델 학습 완료 (TF-IDF 기반).")

In [ ]:
# 2. 테스트 데이터에 대한 예측 수행
y_pred = lr_model.predict(X_test_tfidf)
print("테스트 데이터 예측 완료.")

In [ ]:
# 예측 결과 일부 확인
print("실제 값 (처음 10개):", list(y_test[:10]))
print("예측 값 (처음 10개):", list(y_pred[:10]))

#### ✏️ 연습 문제 (Practice Problems)

1.  로지스틱 회귀 대신, 또 다른 텍스트 분류 모델인 **나이브 베이즈(Naive Bayes)** 모델(`sklearn.naive_bayes.MultinomialNB`)을 사용하여 동일한 데이터로 학습시키고 예측을 수행해 보세요.

In [ ]:
# 코드 작성
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train_tfidf, y_train)

y_pred = nb.predict(X_test_tfidf)

print("실제 값 (처음 10개):", list(y_test[:10]))
print("예측 값 (처음 10개):", list(y_pred[:10]))

2.  내가 직접 작성한 리뷰의 감성을 예측하는 함수 `predict_my_review_tfidf(text)`를 만들어보세요. 함수 내부는 아래 순서를 따라야 합니다.
    * 텍스트 전처리 (`kiwi_preprocess` 함수 사용)
    * 전처리된 텍스트를 TF-IDF 벡터로 변환 (`tfidf_vectorizer.transform` 사용)
    * 학습된 `lr_model`로 예측 (`lr_model.predict` 사용)
    * "이 리뷰는 [긍정/부정]입니다." 라고 결과를 출력

In [ ]:
# 코드 작성

from kiwipiepy import Kiwi

kiwi = Kiwi()


def kiwi_preprocess(text) :
    return " ".join([tok.form for tok in kiwi.tokenize(text, skip_space=True)])

def predict_my_review_tfidf(text) :
    tokenized_text = kiwi_preprocess(text)
    tfidf_matrix = tfidf_vectorizer.transform([tokenized_text])
    label = lr_model.predict(tfidf_matrix)
    if label == 0 :
        return "부정입니다."
    else :
        return "긍정입니다."


---

### 3. 모델 성능 평가하기 (Model Performance Evaluation)

#### 💡 개념 (Concept)

모델이 예측을 얼마나 잘 수행했는지 정량적으로 평가하는 것은 매우 중요합니다. 이를 위해 다양한 **평가 지표**를 사용합니다.

* **정확도 (Accuracy)**: 전체 예측 중 올바르게 예측한 비율. 가장 직관적이지만, 데이터의 클래스(긍정/부정)가 불균형할 경우 성능을 왜곡할 수 있습니다.
* **정밀도 (Precision)**: 모델이 '긍정'으로 예측한 것들 중, 실제 '긍정'인 리뷰의 비율.
* **재현율 (Recall)**: 실제 '긍정'인 리뷰들 중, 모델이 '긍정'으로 예측해낸 비율.
* **F1 점수 (F1-Score)**: 정밀도와 재현율의 조화 평균. 두 지표가 모두 중요할 때 사용하는 균형 잡힌 지표입니다.
* **혼동 행렬 (Confusion Matrix)**: 모델의 예측 결과를 표 형태로 정리한 것. True Positive, False Positive, True Negative, False Negative 값을 통해 모델이 어떤 부분에서 실수를 하는지 직관적으로 파악할 수 있습니다.

#### 💻 예시 코드 (Example Code)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import plotly.figure_factory as ff
import plotly.express as px

# 각 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [ ]:

# 결과를 데이터프레임으로 저장 (나중에 비교 시각화를 위함)
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'TF-IDF': [accuracy, precision, recall, f1]
})

print("--- TF-IDF 기반 모델 성능 ---")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"정밀도 (Precision): {precision:.4f}")
print(f"재현율 (Recall): {recall:.4f}")
print(f"F1 점수 (F1 Score): {f1:.4f}")


In [ ]:
# 혼동 행렬 시각화
conf_matrix = confusion_matrix(y_test, y_pred)
fig = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=['부정(예측)', '긍정(예측)'],
    y=['부정(실제)', '긍정(실제)'],
    colorscale='Blues'
)
fig.update_layout(title_text='<b>혼동 행렬 (Confusion Matrix) - TF-IDF</b>')
fig.show()

---

## Part 2. OpenAI 임베딩을 이용한 최신 AI 감성 분석

### 4. OpenAI 임베딩으로 특징 벡터화하기

#### 💡 개념 (Concept)

TF-IDF는 간단하고 효과적이지만, 단어의 '의미' 자체를 이해하지는 못합니다. 예를 들어, '재미있다'와 '흥미롭다'는 의미가 비슷하지만 TF-IDF는 이들을 완전히 다른 단어로 취급합니다.

**텍스트 임베딩(Text Embedding)**은 이러한 한계를 극복하기 위한 기술입니다. 텍스트를 의미적 관계가 보존된 고차원의 **밀집 벡터(Dense Vector)**로 표현합니다. 이 벡터 공간에서는 의미가 비슷한 단어나 문장들이 서로 가까운 위치에 존재하게 됩니다.

여기서는 `OpenAI`가 제공하는 사전 학습된(Pre-trained) 임베딩 모델(`text-embedding-3-small`)을 사용하여, 우리 데이터의 각 리뷰를 고유한 의미 벡터로 변환하겠습니다. 이렇게 얻은 벡터는 그 자체로 매우 강력한 특징(Feature)이 되어 머신러닝 모델의 성능을 크게 향상시킬 수 있습니다.

#### 💻 예시 코드 (Example Code)

In [ ]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()

# 🚨 중요: OpenAI API 키 설정
# 별도의 설정 파일(.env)을 사용하는 것이 안전합니다.
# https://platform.openai.com/api-keys 에서 키를 발급받을 수 있습니다.
# 본인의 OpenAI API 키를 입력하세요.
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# client = openai.OpenAI() # 환경변수 OPENAI_API_KEY에 키가 등록되어 있으면 자동으로 입력이 됩니다.
client.models.list() # API 키 유효성 검사
print("✅ OpenAI API 키가 성공적으로 설정되었습니다.")


In [44]:
# 텍스트를 임베딩 벡터로 변환하는 함수
def get_embedding(text, model="text-embedding-3-small"):
    # OpenAI 정책에 따라 \n 문자를 공백으로 치환
    text = str(text).replace("\n", " ")
    if not text.strip(): # 비어있거나 공백만 있는 텍스트 처리
        return None
    try:
        return client.embeddings.create(input=[text], model=model).data[0].embedding
    except Exception as e:
        print(f"임베딩 생성 중 오류 발생: {text[:30]}... - {e}")
        return None

In [45]:
# # 학습 데이터와 테스트 데이터에 임베딩 적용
# # 이 과정은 OpenAI API를 호출하므로 인터넷 연결이 필요하며, 데이터 양에 따라 시간이 소요될 수 있습니다.
# print("학습 데이터 임베딩을 시작합니다...")
# train_df['embedding'] = train_df['document_processed'].progress_apply(get_embedding)
# print("\n테스트 데이터 임베딩을 시작합니다...")
# test_df['embedding'] = test_df['document_processed'].progress_apply(get_embedding)

# # 임베딩 생성에 실패한 경우 (예: API 오류, 빈 텍스트) 해당 행 제거
# train_df_emb = train_df.dropna(subset=['embedding'])
# test_df_emb = test_df.dropna(subset=['embedding'])

# print(f"임베딩 생성 완료. {len(train_df) - len(train_df_emb)}개의 학습 데이터, {len(test_df) - len(test_df_emb)}개의 테스트 데이터가 제거되었습니다.")

In [ ]:
# 저장된 임베딩 데이터 로드
import pickle

with open('../datasets/text/nsmc/train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)

with open('../datasets/text/nsmc/test_df.pkl', 'rb') as f:
    test_df = pickle.load(f)

# 임베딩이 없는 행 제거
train_df_emb = train_df.dropna(subset=['embedding'])
test_df_emb = test_df.dropna(subset=['embedding'])

print("저장된 임베딩 데이터를 성공적으로 로드했습니다.")
print(f"로드된 학습 데이터 크기: {len(train_df_emb)}개")
print(f"로드된 테스트 데이터 크기: {len(test_df_emb)}개")

---

### 5. 임베딩 벡터로 모델 학습 및 평가하기

#### 💡 개념 (Concept)

이제 각 리뷰는 1536차원(`text-embedding-3-small` 기준)의 숫자 벡터로 변환되었습니다. 이 임베딩 벡터를 특징(X)으로, 기존의 긍정/부정 레이블(y)을 정답으로 사용하여 새로운 로지스틱 회귀 모델을 학습시키겠습니다.

과연 단어의 의미를 이해하는 임베딩 벡터가 TF-IDF보다 더 나은 성능을 보여줄까요? 동일한 평가 지표를 통해 두 모델의 성능을 직접 비교해봅시다.

#### 💻 예시 코드 (Example Code)

In [ ]:
# 1. 임베딩 벡터와 레이블 분리
X_train_emb = np.array(train_df_emb['embedding'].tolist())
y_train_emb = train_df_emb['label']
X_test_emb = np.array(test_df_emb['embedding'].tolist())
y_test_emb = test_df_emb['label']

print(f"학습 데이터 (Embedding) 형태: {X_train_emb.shape}")
print(f"테스트 데이터 (Embedding) 형태: {X_test_emb.shape}")

In [48]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# 2. 새로운 로지스틱 회귀 모델 생성 및 학습
lr_model_emb = LogisticRegression(random_state=42, C=5, max_iter=1000)
lr_model_emb.fit(X_train_emb, y_train_emb)
print("로지스틱 회귀 모델 학습 완료 (OpenAI Embedding 기반).")

In [ ]:
# 3. 테스트 데이터 예측 및 성능 평가
y_pred_emb = lr_model_emb.predict(X_test_emb)

accuracy_emb = accuracy_score(y_test_emb, y_pred_emb)
precision_emb = precision_score(y_test_emb, y_pred_emb)
recall_emb = recall_score(y_test_emb, y_pred_emb)
f1_emb = f1_score(y_test_emb, y_pred_emb)

# 평가 결과를 기존 데이터프레임에 추가
metrics_df['OpenAI_Embedding'] = [accuracy_emb, precision_emb, recall_emb, f1_emb]

print("--- OpenAI Embedding 기반 모델 성능 ---")
print(metrics_df.to_string())

In [ ]:
# 4. 혼동 행렬 시각화
conf_matrix_emb = confusion_matrix(y_test_emb, y_pred_emb)
fig_emb = ff.create_annotated_heatmap(
    z=conf_matrix_emb,
    x=['부정(예측)', '긍정(예측)'],
    y=['부정(실제)', '긍정(실제)'],
    colorscale='Greens'
)
fig_emb.update_layout(title_text='<b>혼동 행렬 (Confusion Matrix) - OpenAI Embedding</b>')
fig_emb.show()

In [ ]:
# 5. 두 모델 성능 비교 시각화
fig_compare = px.bar(
    metrics_df,
    x='Metric',
    y=['TF-IDF', 'OpenAI_Embedding'],
    barmode='group',
    title='<b>TF-IDF vs OpenAI Embedding 모델 성능 비교</b>',
    labels={'value': 'Score', 'variable': 'Model'},
    text_auto='.4f'
)
fig_compare.show()

#### ✏️ 연습 문제 (Practice Problems)

1.  OpenAI에는 `text-embedding-3-small` 외에 `text-embedding-3-large`와 같은 더 큰 모델이 있습니다. `get_embedding` 함수에서 모델 이름을 변경하여 임베딩을 생성하고, 성능이 어떻게 달라지는지 비교해보세요. (주의: 더 큰 모델은 비용과 시간이 더 많이 소요됩니다.)

In [ ]:
# 200개의 데이터셋만 사용하세요. (학습목적에 맞게 시간과 비용을 줄이기 위함)
train_200_df = train_df.sample(200)
test_200_df = test_df.sample(200)
train_200_df['large_embedding'] = train_200_df['document_processed'].progress_apply(lambda text: get_embedding(text, model="text-embedding-3-large"))
test_200_df['large_embedding'] = test_200_df['document_processed'].progress_apply(lambda text: get_embedding(text, model="text-embedding-3-large"))

train_200_df_emb = train_200_df.dropna(subset=['large_embedding'])
test_200_df_emb = test_200_df.dropna(subset=['large_embedding'])

print(f"로드된 학습 데이터 크기: {len(train_200_df_emb)}개")
print(f"로드된 테스트 데이터 크기: {len(test_200_df_emb)}개")

In [ ]:
large_emb_lr_model = LogisticRegression(random_state=42, C=5, max_iter=1000)

large_emb_lr_model.fit(train_200_df_emb['large_embedding'].tolist(), train_200_df_emb['label'])

y_pred_large_emb = large_emb_lr_model.predict(test_200_df_emb['large_embedding'].tolist())

print(f"Large Embedding 모델 정확도: {accuracy_score(test_200_df_emb['label'], y_pred_large_emb)}")

2.  로지스틱 회귀 모델의 `C` 파라미터는 규제(Regularization)의 강도를 조절합니다. `C` 값을 1, 5, 10으로 변경해가며 임베딩 기반 모델을 다시 학습하고, 각 경우의 정확도가 어떻게 변하는지 확인해보세요. `C` 값이 클수록 규제가 약해집니다.

In [ ]:
# 코드 작성
custom_large_emb_lr_models = {
    f"C{c}": LogisticRegression(random_state=42, C=c, max_iter=1000)
    for c in [1,5,10]
}

for c, model in custom_large_emb_lr_models.items():
    model.fit(train_200_df_emb['large_embedding'].tolist(), train_200_df_emb['label'])
    y_pred_large_emb = model.predict(test_200_df_emb['large_embedding'].tolist())
    print(f"Large Embedding 모델({c}) 정확도: {accuracy_score(test_200_df_emb['label'], y_pred_large_emb)}")


---

## Part 3. 제로샷 분류: 레이블 임베딩 유사도 활용

### 6. 레이블 의미 유사도를 이용한 제로샷(Zero-Shot) 감성 분석

#### 💡 개념 (Concept)

지금까지 우리는 (텍스트 벡터, 레이블) 쌍으로 구성된 학습 데이터를 사용하여 분류 모델(`LogisticRegression`)을 '학습'시켰습니다. 하지만 OpenAI와 같은 강력한 임베딩 모델을 사용하면, 모델을 전혀 학습시키지 않고도 분류를 수행하는 **제로샷(Zero-Shot) 분류**가 가능합니다.

아이디어는 매우 직관적입니다.
1.  분석할 문장 (예: "배우들 연기가 정말 환상적이었어요.")의 임베딩 벡터를 구합니다.
2.  우리가 분류하고 싶은 레이블, 즉 **"긍정"이라는 단어와 "부정"이라는 단어 자체의 임베딩 벡터**를 구합니다.
3.  분석할 문장의 벡터가 '긍정' 벡터와 '부정' 벡터 중 어느 쪽과 더 가까운지(유사한지)를 계산합니다.
4.  더 가까운 쪽 레이블로 문장을 분류합니다.

이 방식은 모델이 텍스트와 레이블의 '의미'를 벡터 공간에서 이해하고 있다는 전제 하에 작동합니다. 별도의 학습 데이터가 필요 없기 때문에 새로운 레이블(예: '중립')을 추가하는 데 매우 유연하다는 엄청난 장점이 있습니다.

벡터 간의 유사도는 주로 **코사인 유사도(Cosine Similarity)**로 측정합니다. 두 벡터가 이루는 각도의 코사인 값으로, 방향이 완전히 같으면 1, 90°로 직교하면 0, 완전히 반대 방향이면 -1의 값을 가집니다.

#### 💻 예시 코드 (Example Code)

In [72]:
from numpy.linalg import norm
from numpy import dot

# 코사인 유사도 계산 함수
def cosine_similarity(A, B):
    if A is None or B is None:
        return -1 # 오류 또는 유효하지 않은 임베딩
    return dot(A, B) / (norm(A) * norm(B))

In [73]:
# 1. 분류할 레이블 정의 및 임베딩
# "긍정", "부정" 보다 좀 더 풍부한 표현이 성능에 도움이 될 수 있습니다.
positive_label_text = "매우 긍정적이고 좋은 평가"
negative_label_text = "매우 부정적이고 나쁜 평가"

positive_embedding = get_embedding(positive_label_text)
negative_embedding = get_embedding(negative_label_text)

In [ ]:
# 2. 제로샷 예측 수행
# 테스트셋의 각 리뷰 임베딩에 대해 레이블 임베딩과의 유사도 계산
y_pred_zero_shot = []
for review_embedding in tqdm(X_test_emb, desc="Zero-Shot 예측 중"):
    sim_positive = cosine_similarity(review_embedding, positive_embedding)
    sim_negative = cosine_similarity(review_embedding, negative_embedding)

    if sim_positive > sim_negative:
        y_pred_zero_shot.append(1) # 긍정
    else:
        y_pred_zero_shot.append(0) # 부정

In [ ]:
# 3. 제로샷 분류 성능 평가
accuracy_zs = accuracy_score(y_test_emb, y_pred_zero_shot)
precision_zs = precision_score(y_test_emb, y_pred_zero_shot)
recall_zs = recall_score(y_test_emb, y_pred_zero_shot)
f1_zs = f1_score(y_test_emb, y_pred_zero_shot)

# 평가 결과를 데이터프레임에 추가
metrics_df['OpenAI_Zero_Shot'] = [accuracy_zs, precision_zs, recall_zs, f1_zs]

print("--- OpenAI Zero-Shot 기반 모델 성능 ---")
print(metrics_df.to_string())

In [77]:
positive_large_embedding = get_embedding(positive_label_text, model='text-embedding-3-large')
negative_large_embedding = get_embedding(negative_label_text, model='text-embedding-3-large')

In [ ]:
y_pred_large_model_zero_shot = []
for review_embedding in tqdm(test_200_df_emb['large_embedding'].tolist(), desc="Zero-Shot 예측 중"):
    sim_positive = cosine_similarity(review_embedding, positive_large_embedding)
    sim_negative = cosine_similarity(review_embedding, negative_large_embedding)

    if sim_positive > sim_negative:
        y_pred_large_model_zero_shot.append(1) # 긍정
    else:
        y_pred_large_model_zero_shot.append(0) # 부정

In [ ]:
# 3. 제로샷 분류 성능 평가
accuracy_zs = accuracy_score(test_200_df_emb['label'].tolist(), y_pred_large_model_zero_shot)
precision_zs = precision_score(test_200_df_emb['label'].tolist(), y_pred_large_model_zero_shot)
recall_zs = recall_score(test_200_df_emb['label'].tolist(), y_pred_large_model_zero_shot)
f1_zs = f1_score(test_200_df_emb['label'].tolist(), y_pred_large_model_zero_shot)

# 평가 결과를 데이터프레임에 추가
metrics_df['OpenAI_Large_Model_Zero_Shot'] = [accuracy_zs, precision_zs, recall_zs, f1_zs]

print("--- OpenAI Zero-Shot 기반 모델 성능 ---")
print(metrics_df.to_string())

In [ ]:
# 4. 혼동 행렬 시각화
conf_matrix_zs = confusion_matrix(y_test_emb, y_pred_zero_shot)
fig_zs = ff.create_annotated_heatmap(
    z=conf_matrix_zs,
    x=['부정(예측)', '긍정(예측)'],
    y=['부정(실제)', '긍정(실제)'],
    colorscale='Oranges'
)
fig_zs.update_layout(title_text='<b>혼동 행렬 (Confusion Matrix) - OpenAI Zero-Shot</b>')
fig_zs.show()

In [ ]:
# 5. 세 가지 모델 성능 비교 시각화
fig_compare_all = px.bar(
    metrics_df,
    x='Metric',
    y=['TF-IDF', 'OpenAI_Embedding', 'OpenAI_Zero_Shot'],
    barmode='group',
    title='<b>세 가지 방식 모델 성능 비교</b>',
    labels={'value': 'Score', 'variable': 'Model'},
    text_auto='.4f'
)
fig_compare_all.show()

#### ✏️ 연습 문제 (Practice Problems)

1.  레이블의 표현 방식은 제로샷 분류 성능에 영향을 미칠 수 있습니다. `positive_label_text`를 '아주 재미있고 훌륭한 영화'와 같이 더 구체적이고 감정이 풍부한 표현으로 바꿔보세요. 부정 레이블도 '지루하고 실망스러운 영화' 등으로 바꾸어 성능이 어떻게 변하는지 확인해보세요.

In [ ]:
# 새로운 레이블 정의
positive_label_enhanced = "아주 재미있고 훌륭한 영화"
negative_label_enhanced = "지루하고 실망스러운 영화"

# 향상된 레이블의 임베딩 계산
positive_embedding_enhanced = get_embedding(positive_label_enhanced)
negative_embedding_enhanced = get_embedding(negative_label_enhanced)

# 향상된 레이블로 제로샷 예측 수행
y_pred_enhanced_zero_shot = []

print("향상된 레이블로 Zero-Shot 예측 중...")
for embedding in tqdm(test_200_df_emb['embedding'].tolist(), desc="향상된 Zero-Shot 예측 중"):
    # 각 레이블과의 코사인 유사도 계산
    pos_similarity = cosine_similarity(embedding, positive_embedding_enhanced)
    neg_similarity = cosine_similarity(embedding, negative_embedding_enhanced)
    
    # 더 높은 유사도를 가진 레이블로 예측
    if pos_similarity > neg_similarity:
        y_pred_enhanced_zero_shot.append(1)  # 긍정
    else:
        y_pred_enhanced_zero_shot.append(0)  # 부정

# 향상된 모델 성능 평가
accuracy_enhanced = accuracy_score(test_200_df_emb['label'].tolist(), y_pred_enhanced_zero_shot)
precision_enhanced = precision_score(test_200_df_emb['label'].tolist(), y_pred_enhanced_zero_shot)
recall_enhanced = recall_score(test_200_df_emb['label'].tolist(), y_pred_enhanced_zero_shot)
f1_enhanced = f1_score(test_200_df_emb['label'].tolist(), y_pred_enhanced_zero_shot)

# 기존 결과와 비교
print("--- 레이블 개선 전후 성능 비교 ---")
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Original_Labels': [accuracy_zs, precision_zs, recall_zs, f1_zs],
    'Enhanced_Labels': [accuracy_enhanced, precision_enhanced, recall_enhanced, f1_enhanced]
})
print(comparison_df.to_string(index=False))

In [ ]:

# 성능 개선 시각화
fig_enhancement = px.bar(
    comparison_df,
    x='Metric',
    y=['Original_Labels', 'Enhanced_Labels'],
    barmode='group',
    title='<b>레이블 개선 전후 Zero-Shot 성능 비교</b>',
    labels={'value': 'Score', 'variable': 'Label Type'},
    text_auto='.4f'
)
fig_enhancement.show()

# 혼동 행렬 비교
conf_matrix_enhanced = confusion_matrix(test_200_df_emb['label'].tolist(), y_pred_enhanced_zero_shot)
fig_enhanced_cm = ff.create_annotated_heatmap(
    z=conf_matrix_enhanced,
    x=['부정(예측)', '긍정(예측)'],
    y=['부정(실제)', '긍정(실제)'],
    colorscale='Greens'
)
fig_enhanced_cm.update_layout(title_text='<b>혼동 행렬 - 향상된 레이블 Zero-Shot</b>')
fig_enhanced_cm.show()

2.  현재는 '긍정'/'부정'의 이진 분류만 수행했습니다. 이 제로샷 기법을 '코미디', '액션', '로맨스', '공포'와 같은 다중 장르 분류 문제에 적용하는 함수 `predict_genre_zero_shot(text)`를 만들어 보세요. 이 함수는 리뷰 텍스트를 입력받아, 4개의 장르 레이블과의 임베딩 유사도를 각각 계산하고 가장 유사도가 높은 장르를 반환해야 합니다.

In [ ]:
# 연습 문제 2: 다중 장르 분류를 위한 제로샷 함수 구현

def predict_genre_zero_shot(text):
    """
    영화 리뷰 텍스트를 입력받아 장르를 예측하는 제로샷 분류 함수
    
    Args:
        text (str): 분류할 영화 리뷰 텍스트
        
    Returns:
        str: 예측된 장르 ('코미디', '액션', '로맨스', '공포')
    """
    if not client:
        return "오류: OpenAI 클라이언트가 설정되지 않았습니다."
    
    # 장르 레이블 정의
    genre_labels = {
        '코미디': '재미있고 유머러스한 코미디 영화',
        '액션': '스릴 넘치고 박진감 있는 액션 영화', 
        '로맨스': '감동적이고 로맨틱한 사랑 영화',
        '공포': '무섭고 긴장감 넘치는 공포 영화'
    }
    
    # 입력 텍스트 전처리
    processed_text = kiwi_preprocess(text)
    
    # 입력 텍스트의 임베딩 계산
    text_embedding = get_embedding(processed_text)
    if text_embedding is None:
        return "오류: 텍스트 임베딩 생성 실패"
    
    # 각 장르 레이블과의 유사도 계산
    similarities = {}
    for genre, label_text in genre_labels.items():
        label_embedding = get_embedding(label_text)
        if label_embedding is not None:
            similarity = cosine_similarity(text_embedding, label_embedding)
            similarities[genre] = similarity
    
    # 가장 높은 유사도를 가진 장르 반환
    if similarities:
        predicted_genre = max(similarities, key=similarities.get)
        return predicted_genre
    else:
        return "오류: 장르 예측 실패"

# 함수 테스트
test_reviews = [
    "정말 웃겨서 배꼽 빠질 뻔했어요. 개그맨들의 연기가 너무 재밌었습니다.",
    "액션 장면이 정말 박진감 넘치고 스턴트가 대단했어요. 손에 땀을 쥐게 하는 영화입니다.",
    "두 주인공의 사랑 이야기가 너무 감동적이었어요. 눈물이 날 정도로 아름다운 로맨스였습니다.",
    "정말 무서워서 잠을 못 잘 것 같아요. 귀신이 나오는 장면에서 소리를 질렀습니다."
]

print("다중 장르 분류 테스트 결과:")
print("-" * 50)
for i, review in enumerate(test_reviews, 1):
    predicted_genre = predict_genre_zero_shot(review)
    print(f"{i}. 리뷰: {review}")
    print(f"   예측 장르: {predicted_genre}")
    print()



---

### 7. 🚀 최종 실습 과제: 나만의 감성 분석 파이프라인 완성하기

지금까지 배운 `Kiwipiepy` 전처리, TF-IDF, 지도 학습 기반 임베딩 모델, 그리고 제로샷 분류까지 모두 종합하여, 새로운 영화 리뷰에 대한 감성을 예측하는 완전한 파이프라인을 구축해봅시다.

아래 `predict_sentiment_pipeline` 함수를 완성하세요. 이 함수는 `method` 인자에 따라 'tfidf', 'openai_logistic', 'openai_zero_shot' 세 가지 방식을 선택하여 예측을 수행해야 합니다.

In [90]:
# 이 셀을 실행하기 전에, 위에서 모든 모델과 벡터라이저가 준비되었다고 가정합니다.

# 제로샷 예측을 위해 레이블 임베딩을 미리 계산해 둡니다.
positive_label_text = "매우 긍정적이고 좋은 평가"
negative_label_text = "매우 부정적이고 나쁜 평가"
positive_embedding = get_embedding(positive_label_text)
negative_embedding = get_embedding(negative_label_text)

In [ ]:
lr_model = LogisticRegression(random_state=42, C=5, max_iter=1000)

lr_model.fit(train_200_df_emb['large_embedding'].tolist(), train_200_df_emb['label'].tolist())

y_pred_large_model = lr_model.predict(test_200_df_emb['large_embedding'].tolist())

print(f"Large Embedding 모델 정확도: {accuracy_score(test_200_df_emb['label'].tolist(), y_pred_large_model)}")

In [ ]:
def predict_sentiment_pipeline(review_text, method='openai_zero_shot'):
    """
    새로운 영화 리뷰의 감성을 예측하는 파이프라인 함수

    Args:
        review_text (str): 감성 분석을 수행할 영화 리뷰 텍스트.
        method (str): 'tfidf', 'openai_logistic', 'openai_zero_shot' 중 사용할 예측 방식을 선택.

    Returns:
        tuple: (예측 레이블(0 or 1), 예측 결과('긍정' or '부정'))
    """
    # 1. 텍스트 전처리
    preprocessed_text = kiwi_preprocess(review_text)

    if method == 'tfidf':
        if 'lr_model' not in globals(): return None, "오류: TF-IDF 모델이 학습되지 않았습니다."
        vectorized_text = tfidf_vectorizer.transform([preprocessed_text])
        prediction = lr_model.predict(vectorized_text)

    elif method == 'openai_logistic':
        if 'lr_model_emb' not in globals(): return None, "오류: 지도학습 임베딩 모델이 학습되지 않았습니다."
        embedding_vector = get_embedding(preprocessed_text)
        if embedding_vector is None: return None, "오류: 임베딩 생성 실패"
        vectorized_text = np.array([embedding_vector])
        prediction = lr_model_emb.predict(vectorized_text)

    elif method == 'openai_zero_shot':
        if not all([positive_embedding, negative_embedding]): return None, "오류: 레이블 임베딩이 준비되지 않았습니다."
        review_embedding = get_embedding(preprocessed_text)
        if review_embedding is None: return None, "오류: 리뷰 임베딩 생성 실패"

        sim_pos = cosine_similarity(review_embedding, positive_embedding)
        sim_neg = cosine_similarity(review_embedding, negative_embedding)

        prediction = np.array([1 if sim_pos > sim_neg else 0])

    else:
        return None, "오류: 'tfidf', 'openai_logistic', 'openai_zero_shot' 방식을 선택해주세요."

    result_text = '긍정' if prediction[0] == 1 else '부정'
    return prediction[0], result_text

# 모델 학습이 완료된 후 아래 코드를 실행하여 테스트해보세요.
if 'lr_model' in globals() and 'lr_model_emb' in globals():
    my_review1 = "이 영화는 정말 시간 가는 줄 모르고 봤네요. 배우들 연기가 일품입니다!"
    my_review2 = "기대하고 봤는데 너무 지루했어요. 스토리가 너무 뻔합니다."

    print("--- 예측 테스트 ---")
    label, text = predict_sentiment_pipeline(my_review1, method='tfidf')
    print(f"리뷰: '{my_review1}' (TF-IDF)\n>> 예측: {text} ({label})\n")

    label, text = predict_sentiment_pipeline(my_review1, method='openai_logistic')
    print(f"리뷰: '{my_review1}' (지도학습 임베딩)\n>> 예측: {text} ({label})\n")

    label, text = predict_sentiment_pipeline(my_review1, method='openai_zero_shot')
    print(f"리뷰: '{my_review1}' (제로샷 임베딩)\n>> 예측: {text} ({label})\n")

    label, text = predict_sentiment_pipeline(my_review2, method='openai_zero_shot')
    print(f"리뷰: '{my_review2}' (제로샷 임베딩)\n>> 예측: {text} ({label})")
else:
    print("🚨 모델이 아직 학습되지 않았습니다. 이전 코드 셀들을 먼저 실행해주세요.")